In [ ]:
#Import libraries
import torch
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm
import zipfile
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pickle as pkl
import matplotlib.pyplot as plt
from torchvision import transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Upload dataset

df_training = pd.read_csv('drive/MyDrive/training.csv', sep="\t") # Put your path to training.csv here
df_test = pd.read_csv('drive/MyDrive/trial.csv', sep="\t") # Put your path to trail.csv here

In [ ]:
#CNN Model

class Binary_Classifier(nn.Module):
    def __init__(self):
        super(Binary_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(58320, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#hyper parameters

learning_rate = 0.01

# Initialize Model , Optimizer, Loss

model = Binary_Classifier().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [ ]:
# Due to the large input, the model is being trained in steps to avoid memory collapsing. It has been trained on the trial and training dataset.

#For the training set

steps = 10
step_training=int(df_training.shape[0]/steps)
step_test=int(df_test.shape[0]/steps)

for step in range(steps):
  input_training = []
  label_training = []
  img_zip = zipfile.ZipFile('drive/MyDrive/training.zip') # put you path to training.zip here
  img_zip.setpassword(b'*MaMiSemEval2022!')

  for i, row in tqdm(df_training.iloc[step_training*(step):step_training*(step+1)].iterrows()):
      try:
          filename = row['file_name']
          convert_tensor = transforms.Compose([
          transforms.Resize((224, 224)),
          transforms.ToTensor()
      ])
          img = Image.open(img_zip.open('TRAINING/' + filename)) 
          #img = img.resize((39, 39), Image.ANTIALIAS)
          tensor=convert_tensor(img)
          if tensor.shape[0] == 3:
              input_training.append(tensor)
              label_training.append(torch.tensor(row['misogynous'],dtype=torch.long))
      except:
          print('........................')
          continue

  #For the trial set
  
  img_zip = zipfile.ZipFile('drive/MyDrive/trial.zip') # put your path to trial.zip here
  img_zip.setpassword(b'*MaMiSemEval2022!')
  for i, row in tqdm(df_test.iloc[step_test*step:step_test*(step+1)].iterrows()):
      try:
          filename = row['file_name']
          convert_tensor = transforms.Compose([
          transforms.Resize((224, 224)),
          transforms.ToTensor()
      ])
          img = Image.open(img_zip.open('Users/fersiniel/Desktop/MAMI - TO LABEL/TRIAL DATASET/' + filename)) 
          #img = img.resize((39, 39), Image.ANTIALIAS)
          tensor=convert_tensor(img)
          if tensor.shape[0] == 3:
              input_training.append(tensor)
              label_training.append(torch.tensor(row['misogynous'],dtype=torch.long))
      except Exception as E:
          
          print(E)
          continue

  train_tensor = torch.stack(input_training,0).to(device=device) 
  label_tensor = torch.stack(label_training,0).to(device=device)
  del input_training, label_training
  input, input_t, label, label_t = train_test_split(train_tensor, label_tensor, test_size=0.2)
  del train_tensor, label_tensor

  
  class dataset(Dataset):
    def __init__(self,x,y):
      self.x = x
      self.y = y
      self.length = self.x.shape[0]

    def __getitem__(self,idx):
      return self.x[idx],self.y[idx]
    def __len__(self):
      return self.length
  trainset = dataset(input, label)
 
  #DataLoader
  
  trainloader = DataLoader(trainset,batch_size=32,shuffle=True)


  #forward loop
 
  losses = []
  accur = []
  epochs = 5
  acc_val=[]
  for i in tqdm(range(epochs)):
      for j,(x_train,y_train) in enumerate(trainloader):

          #calculate output
         
          output = model(x_train)

          #calculate loss
          #loss = loss_fn(output,y_train.reshape(-1,1))
          
          loss = loss_fn(output.reshape(-1,2),y_train)


          #backprop
          
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          #accuracy
          
          if j%10 == 0:
              with torch.no_grad():
                  losses.append(loss)
                  acc =(label == torch.argmax(model(input),dim=1)).sum()/label.size()[0]
                  acc_t = (label_t == torch.argmax(model(input_t),dim=1)).sum()/label_t.size()[0]
                  accur.append(acc)
                  acc_val.append(acc_t)
                  print(f"\nstep: {step}, epoch {i}, loss : {loss}, training accuracy : {acc}, test accuracy : {acc_t}")

1000it [00:11, 85.74it/s]
10it [00:04,  2.06it/s]
  0%|          | 0/5 [00:00<?, ?it/s]


step: 0, epoch 0, loss : 0.6926514506340027, training accuracy : 0.521039605140686, test accuracy : 0.5247524380683899

step: 0, epoch 0, loss : 0.8841594457626343, training accuracy : 0.4801980257034302, test accuracy : 0.47524750232696533


 20%|██        | 1/5 [00:01<00:04,  1.12s/it]


step: 0, epoch 0, loss : 0.6653885245323181, training accuracy : 0.6509900689125061, test accuracy : 0.608910858631134

step: 0, epoch 1, loss : 0.6690996289253235, training accuracy : 0.6361386179924011, test accuracy : 0.608910858631134

step: 0, epoch 1, loss : 0.610276460647583, training accuracy : 0.6262376308441162, test accuracy : 0.594059407711029


 40%|████      | 2/5 [00:02<00:02,  1.02it/s]

...


step: 9, epoch 2, loss : 0.14858964085578918, training accuracy : 0.9987608790397644, test accuracy : 0.5841584205627441

step: 9, epoch 3, loss : 0.03426000475883484, training accuracy : 0.997521698474884, test accuracy : 0.5891088843345642

step: 9, epoch 3, loss : 0.03056998737156391, training accuracy : 0.9962825775146484, test accuracy : 0.5693069100379944


 80%|████████  | 4/5 [00:03<00:00,  1.15it/s]


step: 9, epoch 3, loss : 0.03206696733832359, training accuracy : 0.9987608790397644, test accuracy : 0.5891088843345642

step: 9, epoch 4, loss : 0.008630191907286644, training accuracy : 1.0, test accuracy : 0.5693069100379944

step: 9, epoch 4, loss : 0.0074251797050237656, training accuracy : 0.9987608790397644, test accuracy : 0.5445544719696045


100%|██████████| 5/5 [00:04<00:00,  1.15it/s]


step: 9, epoch 4, loss : 0.01673254370689392, training accuracy : 1.0, test accuracy : 0.5841584205627441


# Test

In [ ]:
#Upoad and read test data
img_zip = zipfile.ZipFile('test.zip') # put your path to test.zip here
img_zip.setpassword(b'*MaMiSemEval2022!')
df_test = pd.read_csv('test_labels.txt', delimiter="\t", header=None) # put your path to test_labels.txt here

In [ ]:
#Pre-processing test data
input = []
label = []

for i, row in tqdm(df_test.iterrows()):
    try:
        filename = row['file_name']
        convert_tensor = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

         
        
        img = Image.open(img_zip.open('test/' + filename)) 
        
        tensor=convert_tensor(img)
        if tensor.shape[0] == 3:
            input.append(tensor)
            label.append(torch.tensor(row['misogynous'],dtype=torch.long))
    except Exception as e:
        print(e)
        continue

1000it [04:08,  4.02it/s]


In [ ]:
test_tensor = torch.stack(input,0).to(device=device) 
test_label_tensor = torch.stack(label,0).to(device=device)

In [ ]:
output = torch.argmax(model(test_tensor),dim=1)

In [ ]:
output.shape

torch.Size([1000])

In [ ]:
#Create a pickle file of the predicted labels of the cnn to be used for the fusion

arrayInput = output #Trial input
fileName = "cnn_output.pkl"
fileObject = open(fileName, 'wb')
pkl.dump(arrayInput, fileObject)
fileObject.close()